In [ ]:
from ipython_tools import handler
handler.style()

# Simulate cosmic events

In [ ]:
import basf2
from cdc.cr import add_cdc_cr_simulation, set_cdc_cr_parameters

set_cdc_cr_parameters("normal")

path = basf2.create_path()

path.add_module("EventInfoSetter", evtNumList=[5000])

path.add_module('Gearbox', override=[
    ("/Global/length", "20", "m"),
    ("/Global/width", "20", "m"),
    ("/Global/height", "9", "m")]
)
path.add_module("Geometry", components=["CDC"])

add_cdc_cr_simulation(path, empty_path=basf2.create_path(), topInCounter=False)

path.add_module("RootOutput", outputFileName="simulated_cosmics.root")

calculation = handler.process(path)

In [ ]:
calculation.start()
calculation.wait_for_end()

# Analyse the created cosmic events

In [ ]:
import basf2
import tracking

from tracking.harvesting_validation.combined_module import CombinedTrackingValidationModule

from trackfindingcdc.cdcdisplay import CDCSVGDisplayModule

In [ ]:
import os 
try:
    os.mkdir("results")
except OSError:
    pass

def create_path(merge):
    output_file = "results/validation_cosmics_{merge}.root".format(merge=merge)
    
    if os.path.exists(output_file):
        return {"path": None}
    
    path = basf2.create_path()
    
    path.add_module("RootInput", inputFileName="simulated_cosmics.root")

    path.add_module('Gearbox', override=[
        ("/Global/length", "20", "m"),
        ("/Global/width", "20", "m"),
        ("/Global/height", "9", "m")]
    )
    path.add_module("Geometry", components=["CDC"])
    
    tracking.add_cdc_cr_track_finding(path, merge_tracks=merge)
    
    display_module = CDCSVGDisplayModule(output_folder="pictures_{merge}".format(merge=merge), interactive=False)
    display_module.draw_recotracks = True
    path.add_module(display_module)
    
    tracking.add_mc_matcher(path)
    
    path.add_module(CombinedTrackingValidationModule(expert_level=200, output_file_name=output_file, name="", contact="."))
    
    return {"path": path}

calculation = handler.process_parameter_space(create_path, merge=[True, False])


In [ ]:
calculation.start()
calculation.wait_for_end()

In [ ]:
hists = ["MCSideTrackingValidationModule_overview_figures_of_merit", "PRSideTrackingValidationModule_overview_figures_of_merit"]

In [ ]:
import uproot
df = uproot.concatenate([f"results/validation_cosmics_True.root:{hist}" for hist in hists], library="pd", axis=1)

In [ ]:
df = uproot.concatenate([f"results/validation_cosmics_False.root:{hist}" for hist in hists], library="pd", axis=1)